In [1]:
import potholes_dataset
import tensorflow as tf
import time
from datetime import timedelta
import math
import random
import numpy as np
import os
import pandas as pd

/Users/omsoni/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
os.getcwd()

'/Users/omsoni/tensorflow/models/tutorials/image/imagenet/cv-tricks.com/Tensorflow-tutorials/tutorial-2-image-classifier'

In [3]:
batch_size = 5
train_path = './data/Subset1-Simplex/'
filename = 'simpleTrainAll.csv'

#Prepare input data
classes = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])
#classes = np.array2string(classes)
print(classes)
num_classes = len(classes)
# 20% of the data will automatically be used for validation
validation_size = 0.2
img_size = 128
num_channels = 3

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]


In [4]:
metadata_file = os.path.join(train_path, filename)
#'./cv-tricks.com/Tensorflow-tutorials/tutorial-2-image-classifier/data/Subset1-Simplex/simpleTrainFullPhotosSortedFullAnnotations.csv'
df = pd.read_csv(metadata_file, header=None)
#path = "./cv-tricks.com/Tensorflow-tutorials/tutorial-2-image-classifier/data/Subset1-Simplex/"
#print(df.head(5))
files = df.iloc[:,0].map(str)
labels = df.iloc[:,1].values
#print('labels',labels)
classes = pd.unique(labels)
#print('classes', classes)

In [5]:
#Adding Seed so that random initialization is consistent
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [6]:
# We shall load all the training and validation images and labels into memory using openCV and use that during training
data = potholes_dataset.read_train_sets(train_path, filename, img_size, validation_size=validation_size)
#print('data {0}'.format(data.train.images))

session = tf.Session()
x = tf.placeholder(tf.float32, shape=[None, img_size,img_size,num_channels], name='x')

## labels
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)


                                 0   1     2     3    4   5       6       7   \
0  Train-data\Positive\G0010033.jpg   6  1990  1406   66  14  1464.0  1442.0   
1  Train-data\Positive\G0010034.jpg   5  1804  1424   90  16  1192.0  1484.0   
2  Train-data\Positive\G0010035.jpg   4  1590  1452  106  24   840.0  1544.0   
3  Train-data\Positive\G0010036.jpg   3  1322  1516  134  28   362.0  1658.0   
4  Train-data\Positive\G0010037.jpg   2   884  1638  190  52  1428.0  1654.0   

      8     9  ...  48  49  50  51  52  53  54  55  56  57  
0   92.0  16.0 ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
1  110.0  16.0 ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
2  132.0  28.0 ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
3  156.0  38.0 ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  
4   98.0  26.0 ... NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  

[5 rows x 58 columns]
labels [6 5 4 ... 0 0 0]
classes [ 6  5  4  3  2  7  9  8  1 10 11 14 12 13  0]
Instructions for updating:
Use the `axis` argument

In [7]:
##Network graph params
# Filter size 3x3
filter_size_conv1 = 3 

# 32 filters = 32 features ?
num_filters_conv1 = 32

# same as in layer 1
filter_size_conv2 = 3
num_filters_conv2 = 32

# Same as in layer 2 but more filters why?
filter_size_conv3 = 3
num_filters_conv3 = 64
    
fc_layer_size = 128

In [8]:
def create_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def create_biases(size):
    return tf.Variable(tf.constant(0.05, shape=[size]))


## Convulational Layer
def create_convolutional_layer(input,
               num_input_channels, 
               conv_filter_size,        
               num_filters):  
    
    ## We shall define the weights that will be trained using create_weights function.
    weights = create_weights(shape=[conv_filter_size, conv_filter_size, num_input_channels, num_filters])
    ## We create biases using the create_biases function. These are also trained.
    biases = create_biases(num_filters)

    ## Creating the convolutional layer
    layer = tf.nn.conv2d(input=input,
                     filter=weights,
                     strides=[1, 1, 1, 1],
                     padding='SAME')

    layer += biases

    ## We shall be using max-pooling.  
    layer = tf.nn.max_pool(value=layer,
                            ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')
    ## Output of pooling is fed to Relu which is the activation function for us.
    layer = tf.nn.relu(layer)

    return layer


In [9]:
def create_flatten_layer(layer):
    #We know that the shape of the layer will be [batch_size img_size img_size num_channels] 
    # But let's get it from the previous layer.
    layer_shape = layer.get_shape()

    ## Number of features will be img_height * img_width* num_channels. But we shall calculate it in place of hard-coding it.
    num_features = layer_shape[1:4].num_elements()

    ## Now, we Flatten the layer so we shall have to reshape to num_features
    layer = tf.reshape(layer, [-1, num_features])

    return layer

## Fully Connected Layer
def create_fc_layer(input,          
             num_inputs,    
             num_outputs,
             use_relu=True):
    
    #Let's define trainable weights and biases.
    weights = create_weights(shape=[num_inputs, num_outputs])
    biases = create_biases(num_outputs)

    # Fully connected layer takes input x and produces wx+b.Since, these are matrices, we use matmul function in Tensorflow
    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer

In [10]:
layer_conv1 = create_convolutional_layer(input=x,
               num_input_channels=num_channels,
               conv_filter_size=filter_size_conv1,
               num_filters=num_filters_conv1)
layer_conv2 = create_convolutional_layer(input=layer_conv1,
               num_input_channels=num_filters_conv1,
               conv_filter_size=filter_size_conv2,
               num_filters=num_filters_conv2)

layer_conv3= create_convolutional_layer(input=layer_conv2,
               num_input_channels=num_filters_conv2,
               conv_filter_size=filter_size_conv3,
               num_filters=num_filters_conv3)
          
layer_flat = create_flatten_layer(layer_conv3)

layer_fc1 = create_fc_layer(input=layer_flat,
                     num_inputs=layer_flat.get_shape()[1:4].num_elements(),
                     num_outputs=fc_layer_size,
                     use_relu=True)

layer_fc2 = create_fc_layer(input=layer_fc1,
                     num_inputs=fc_layer_size,
                     num_outputs=num_classes,
                     use_relu=False) 

y_pred = tf.nn.softmax(layer_fc2,name='y_pred')

y_pred_cls = tf.argmax(y_pred, dimension=1)
session.run(tf.global_variables_initializer())
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,
                                                    labels=y_true)
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


session.run(tf.global_variables_initializer()) 

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [11]:
def show_progress(epoch, feed_dict_train, feed_dict_validate, val_loss):
    acc = session.run(accuracy, feed_dict=feed_dict_train)
    val_acc = session.run(accuracy, feed_dict=feed_dict_validate)
    msg = "Training Epoch {0} --- Training Accuracy: {1:>6.1%}, Validation Accuracy: {2:>6.1%},  Validation Loss: {3:.3f}"
    print(msg.format(epoch + 1, acc, val_acc, val_loss))

total_iterations = 0

saver = tf.train.Saver()

In [12]:
def train(num_iteration):
    global total_iterations
    
    for i in range(total_iterations,
                   total_iterations + num_iteration):

        x_batch, y_true_batch,imgNames_batch = data.train.next_batch(batch_size)
        #print('x_batch {0}, y_true_batch {1},imgNames_batch{2}', x_batch, y_true_batch,imgNames_batch)
        x_valid_batch, y_valid_batch, imgNames_batch = data.valid.next_batch(batch_size)

        
        feed_dict_tr = {x: x_batch,
                           y_true: y_true_batch}
        #print('feed_dict_tr{0}'.format(feed_dict_tr))
        feed_dict_val = {x: x_valid_batch,
                              y_true: y_valid_batch}

        session.run(optimizer, feed_dict=feed_dict_tr)

        if i % int(data.train.num_examples/batch_size) == 0: 
            val_loss = session.run(cost, feed_dict=feed_dict_val)
            epoch = int(i / int(data.train.num_examples/batch_size))    
            
            show_progress(epoch, feed_dict_tr, feed_dict_val, val_loss)
            saver.save(session, './SavedModel/multiclass_potholes_model') 


    total_iterations += num_iteration


In [14]:
train(num_iteration=100)

Training Epoch 1 --- Training Accuracy: 100.0%, Validation Accuracy:  80.0%,  Validation Loss: 5.957
Training Epoch 2 --- Training Accuracy: 100.0%, Validation Accuracy:  80.0%,  Validation Loss: 5.945
Training Epoch 3 --- Training Accuracy: 100.0%, Validation Accuracy:  80.0%,  Validation Loss: 6.011
Training Epoch 4 --- Training Accuracy: 100.0%, Validation Accuracy:  80.0%,  Validation Loss: 6.003
Training Epoch 5 --- Training Accuracy: 100.0%, Validation Accuracy:  80.0%,  Validation Loss: 6.033
Training Epoch 6 --- Training Accuracy: 100.0%, Validation Accuracy:  80.0%,  Validation Loss: 6.057
Training Epoch 7 --- Training Accuracy: 100.0%, Validation Accuracy:  80.0%,  Validation Loss: 6.068
Training Epoch 8 --- Training Accuracy: 100.0%, Validation Accuracy:  80.0%,  Validation Loss: 6.102
Training Epoch 9 --- Training Accuracy: 100.0%, Validation Accuracy:  80.0%,  Validation Loss: 6.118
Training Epoch 10 --- Training Accuracy: 100.0%, Validation Accuracy:  80.0%,  Validation L